In [3]:
# =====================================================
# FAKE NEWS DETECTION – (RoBERTa-base version) + FAKENEWSNET
# =====================================================

# 1. INSTALL & IMPORT
!pip install -q transformers datasets torch scikit-learn pandas numpy psutil accelerate

import os, re, shutil, psutil, warnings
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, Dataset, DatasetDict, concatenate_datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    Trainer, TrainingArguments, EarlyStoppingCallback,
    DataCollatorWithPadding
)
from google.colab import drive

warnings.filterwarnings("ignore")

# 2. GPU INFO
if torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
else:
    device_name = "CPU"
    vram_gb = 0.0

print(f"Device: {device_name} | CUDA: {torch.cuda.is_available()} | VRAM: {vram_gb:.1f} GB")

# 3. MOUNT DRIVE
if not os.path.exists('/content/drive'):
    try:
        drive.mount('/content/drive', force_remount=True)
    except ValueError:
        print("Drive có thể đã được mount. Bỏ qua.")

OUTPUT_DIR = "/content/drive/MyDrive/FakeNewsNet_RoBERTa_base_Pro"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# 4. TẢI DATASET FAKENEWSNET & PRE-PROCESSING
print("\n⏳ Đang tải dataset FakeNewsNet...")

# FakeNewsNet thường chia làm 2 tập con: gossipcop và politifact
try:
    ds_gossip = load_dataset("rickstello/FakeNewsNet", "gossipcop", split="train")
    ds_politi = load_dataset("rickstello/FakeNewsNet", "politifact", split="train")
    dataset_full = concatenate_datasets([ds_gossip, ds_politi])
    df = pd.DataFrame(dataset_full)
except Exception as e:
    print(f"⚠️ Tải config con thất bại ({e}), tải bản default...")
    dataset = load_dataset("rickstello/FakeNewsNet", split="train")
    df = pd.DataFrame(dataset)

print(f"Tổng số mẫu ban đầu: {len(df)}")

# Chuẩn hóa tên cột (Dataset này tên cột hơi lộn xộn tùy phiên bản)
text_col = next((c for c in ['news_content', 'text', 'content', 'body'] if c in df.columns), None)
title_col = next((c for c in ['title', 'news_title', 'headline'] if c in df.columns), None)
label_col = next((c for c in ['real', 'label', 'class', 'fake'] if c in df.columns), None)

if not label_col: raise ValueError("❌ Không tìm thấy cột nhãn!")

df['label'] = df[label_col].astype(int)

# --- CLEANING CHO ROBERTA (Giữ Case, Clean riêng lẻ) ---
def clean_text(s):
    if not isinstance(s, str): return ""
    # KHÔNG dùng .lower()

    s = re.sub(r'https?://\S+', ' ', s) # Bỏ URL
    s = re.sub(r'<.*?>', ' ', s)        # Bỏ HTML

    # Giữ lại chữ hoa, số và dấu câu cơ bản
    s = re.sub(r'[^a-zA-Z0-9\s\.\,\!\?]', ' ', s)

    s = re.sub(r'\s+', ' ', s).strip()
    return s

print("🧹 Đang làm sạch dữ liệu (RoBERTa style)...")

# Clean từng cột trước
df['title_clean'] = df[title_col].fillna('').apply(clean_text) if title_col else ""
df['text_clean'] = df[text_col].fillna('').apply(clean_text) if text_col else ""

# Ghép chuỗi dùng Separator </s>
df['content'] = df['title_clean'] + " </s> " + df['text_clean']

# Lọc bỏ mẫu quá ngắn
df = df[df['content'].str.len() > 20].drop_duplicates(subset=['content'])
print(f" → Sau xử lý: {len(df):,}")

# Class Weights (Quan trọng vì FakeNewsNet mất cân bằng)
classes = np.unique(df['label'])
class_weights = compute_class_weight('balanced', classes=classes, y=df['label'])
# Map về đúng index nếu label không tuần tự (dù dataset này thường là 0,1)
class_weight_dict = {i: float(w) for i, w in zip(classes, class_weights)}
print("Class weights:", class_weight_dict)

# 5. SPLIT DATA
# Chia stratified để giữ tỷ lệ nhãn
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df['label'])
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42, stratify=temp_df['label'])

dataset_dict = DatasetDict({
    "train": Dataset.from_pandas(train_df[['content','label']].reset_index(drop=True)),
    "validation": Dataset.from_pandas(val_df[['content','label']].reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df[['content','label']].reset_index(drop=True))
})

# 6. TOKENIZER (RoBERTa-BASE)
MODEL_NAME = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_fn(batch):
    return tokenizer(batch["content"], truncation=True, max_length=384, padding=False)

print("⚙️ Tokenizing...")
tokenized = dataset_dict.map(tokenize_fn, batched=True, batch_size=1000, remove_columns=['content'])
tokenized = tokenized.rename_column("label", "labels")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 7. MODEL SETUP
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
# Config nhãn (0: Fake, 1: Real - Dựa trên logic cột 'real' của dataset gốc)
model.config.id2label = {0: "Fake", 1: "Real"}
model.config.label2id = {"Fake": 0, "Real": 1}

# 8. CHECKPOINT MANAGEMENT
def get_last_checkpoint(output_dir):
    if not os.path.exists(output_dir): return None
    ckpts = [c for c in os.listdir(output_dir) if c.startswith("checkpoint-")]
    if not ckpts: return None
    # Sort theo số checkpoint
    ckpts_sorted = sorted(ckpts, key=lambda x: int(x.split('-')[-1]), reverse=True)
    return os.path.join(output_dir, ckpts_sorted[0])

last_checkpoint = get_last_checkpoint(OUTPUT_DIR)
print(f"Checkpoint: {last_checkpoint if last_checkpoint else 'Training from scratch'}")

# 9. CUSTOM TRAINER (Weighted Loss)
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        # Convert weight dict to tensor list chuẩn
        weights = [class_weight_dict[i] for i in sorted(class_weight_dict.keys())]
        w = torch.tensor(weights, dtype=torch.float32, device=model.device)
        loss_fct = torch.nn.CrossEntropyLoss(weight=w)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

# 10. TRAINING ARGS
training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=2,
    learning_rate=3e-5,                # RoBERTa thích LR thấp
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted", zero_division=0)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)

# 11. TRAIN
print("\n🚀 BẮT ĐẦU HUẤN LUYỆN ROBERTA-BASE (FAKENEWSNET)...")
trainer.train(resume_from_checkpoint=last_checkpoint)

# 12. EVALUATE & SAVE
print("\n🎯 KẾT QUẢ TRÊN TEST SET:")
print(trainer.evaluate(tokenized["test"]))

final_path = os.path.join(OUTPUT_DIR, "final_roberta_fnn")
trainer.save_model(final_path)
tokenizer.save_pretrained(final_path)

# Dọn dẹp checkpoint thừa
for f in os.listdir(OUTPUT_DIR):
    if f.startswith("checkpoint-"):
        shutil.rmtree(os.path.join(OUTPUT_DIR, f), ignore_errors=True)

print(f"\n✅ Đã lưu model tại: {final_path}")

Device: Tesla T4 | CUDA: True | VRAM: 15.8 GB

⏳ Đang tải dataset FakeNewsNet...
⚠️ Tải config con thất bại (BuilderConfig 'gossipcop' not found. Available: ['default']), tải bản default...
Tổng số mẫu ban đầu: 23196
🧹 Đang làm sạch dữ liệu (RoBERTa style)...
 → Sau xử lý: 21,388
Class weights: {np.int64(0): 2.0668728256667954, np.int64(1): 0.6595534723078821}
⚙️ Tokenizing...


Map:   0%|          | 0/17110 [00:00<?, ? examples/s]

Map:   0%|          | 0/2139 [00:00<?, ? examples/s]

Map:   0%|          | 0/2139 [00:00<?, ? examples/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Checkpoint: Training from scratch

🚀 BẮT ĐẦU HUẤN LUYỆN ROBERTA-BASE (FAKENEWSNET)...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.456500,0.555396,0.848995,0.836086,0.843575,0.848995
2,0.367400,0.421359,0.827957,0.834406,0.848576,0.827957
3,0.280000,0.497071,0.847592,0.850903,0.856484,0.847592



🎯 KẾT QUẢ TRÊN TEST SET:


{'eval_loss': 0.4734254777431488, 'eval_accuracy': 0.8457223001402524, 'eval_f1': 0.8501305428140348, 'eval_precision': 0.8587566375235793, 'eval_recall': 0.8457223001402524, 'eval_runtime': 3.1709, 'eval_samples_per_second': 674.565, 'eval_steps_per_second': 42.259, 'epoch': 3.0}

✅ Đã lưu model tại: /content/drive/MyDrive/FakeNewsNet_RoBERTa_base_Pro/final_roberta_fnn
